In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

Using TensorFlow backend.


In [2]:
base = pd.read_csv('../data/autos.csv', encoding = 'ISO-8859-1')

# Pré Processamento

In [3]:
base = base.drop('dateCrawled', axis = 1)
base = base.drop('dateCreated', axis = 1)
base = base.drop('nrOfPictures', axis = 1)
base = base.drop('postalCode', axis = 1)
base = base.drop('lastSeen', axis = 1)

In [4]:
base['name'].value_counts()

Ford_Fiesta                                                           657
BMW_318i                                                              627
Opel_Corsa                                                            622
Volkswagen_Golf_1.4                                                   603
BMW_316i                                                              523
BMW_320i                                                              492
Volkswagen_Polo                                                       475
Renault_Twingo                                                        447
Volkswagen_Golf                                                       428
Volkswagen_Golf_1.6                                                   413
Volkswagen_Polo_1.2                                                   412
BMW_116i                                                              394
Opel_Corsa_1.2_16V                                                    373
Opel_Corsa_B                          

In [5]:
base = base.drop('name', axis = 1)

In [6]:
base['seller'].value_counts()

privat        371525
gewerblich         3
Name: seller, dtype: int64

In [7]:
base = base.drop('seller', axis = 1)

In [8]:
base['offerType'].value_counts()

Angebot    371516
Gesuch         12
Name: offerType, dtype: int64

In [9]:
base = base.drop('offerType', axis = 1)

# Variáveis inconsistentes

In [10]:
#preços abaixo de 10 reais
i1 = base.loc[base.price <= 10]
base.price.mean()
base = base[base.price > 10]

In [11]:
#preços inconsistentes, geralmente será 999.999 ou 11.111.111
i2 = base.loc[base.price > 350000]
base = base.loc[base.price < 350000]

In [12]:
base.isnull().sum()

price                      0
abtest                     0
vehicleType            33546
yearOfRegistration         0
gearbox                17236
powerPS                    0
model                  17967
kilometer                  0
monthOfRegistration        0
fuelType               29391
brand                      0
notRepairedDamage      65986
dtype: int64

In [13]:
base['vehicleType'].value_counts() # limousine

limousine     93614
kleinwagen    78014
kombi         65921
bus           29699
cabrio        22509
coupe         18386
suv           14477
andere         3125
Name: vehicleType, dtype: int64

In [14]:
base['gearbox'].value_counts() # manuell

manuell      266547
automatik     75508
Name: gearbox, dtype: int64

In [15]:
base['model'].value_counts() # golf

golf                  28989
andere                25560
3er                   19905
polo                  12604
corsa                 12149
astra                 10458
a4                     9972
passat                 9966
c_klasse               8624
5er                    8304
e_klasse               7379
a3                     6498
a6                     5832
focus                  5805
fiesta                 5645
transporter            5371
twingo                 4804
2_reihe                4753
fortwo                 4286
a_klasse               4241
vectra                 4072
1er                    3818
mondeo                 3503
touran                 3492
clio                   3456
3_reihe                3427
punto                  3238
zafira                 2983
megane                 2809
ibiza                  2634
                      ...  
mii                      81
crossfire                66
range_rover_evoque       65
gl                       63
nubira              

In [16]:
base['fuelType'].value_counts() # benzin

benzin     217582
diesel     106002
lpg          5222
cng           557
hybrid        271
andere        165
elektro       101
Name: fuelType, dtype: int64

In [17]:
base['notRepairedDamage'].value_counts() # nein

nein    259301
ja       34004
Name: notRepairedDamage, dtype: int64

In [18]:
valores = {'vehicleType': 'limousine', 'gearbox': 'manuell',
           'model': 'golf', 'fuelType': 'benzin',
           'notRepairedDamage': 'nein'}
base = base.fillna(value = valores)

In [19]:
previsores = base.iloc[:, 1:13].values
preco_real = base.iloc[:, 0].values

# One Hot Encoder

In [20]:
labelencoder_previsores = LabelEncoder()
previsores[:, 0] = labelencoder_previsores.fit_transform(previsores[:, 0])
previsores[:, 1] = labelencoder_previsores.fit_transform(previsores[:, 1])
previsores[:, 3] = labelencoder_previsores.fit_transform(previsores[:, 3])
previsores[:, 5] = labelencoder_previsores.fit_transform(previsores[:, 5])
previsores[:, 8] = labelencoder_previsores.fit_transform(previsores[:, 8])
previsores[:, 9] = labelencoder_previsores.fit_transform(previsores[:, 9])
previsores[:, 10] = labelencoder_previsores.fit_transform(previsores[:, 10])

In [ ]:
onehotencoder = OneHotEncoder(categorical_features = [0,1,3,5,8,9,10])
previsores = onehotencoder.fit_transform(previsores).toarray()

/home/eduardo/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:414: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/eduardo/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:450: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [ ]:
regressor = Sequential()
regressor.add(Dense(units = 158, activation = 'relu', input_dim = 316))
regressor.add(Dense(units = 158, activation = 'relu'))
regressor.add(Dense(units = 1, activation = 'linear'))
regressor.compile(loss = 'mean_absolute_error', optimizer = 'adam',
                  metrics = ['mean_absolute_error'])
regressor.fit(previsores, preco_real, batch_size = 300, epochs = 100)

In [ ]:
previsoes = regressor.predict(previsores)

In [ ]:
preco_real.mean()

In [ ]:
previsoes.mean()